In [1]:
from __future__ import annotations

import argparse
from logging import INFO, FileHandler, getLogger

import yaml

from config.config import Config
from src.data_collaboration import DataCollaborationAnalysis
from src.load_data import load_data # , load_data_breast_cancer, load_data_diabetes, load_data_har
from src.paths import CONFIG_DIR, INPUT_DIR, OUTPUT_DIR
from src.institutional_analysis import centralize_analysis, dca_analysis, individual_analysis

import sys
sys.argv = ["notebook", "exp004"]

parser = argparse.ArgumentParser()
parser.add_argument("name", type=str)
args = parser.parse_args()

# 引数の設定
parser = argparse.ArgumentParser()
parser.add_argument("name", type=str, default="exp004")
args = parser.parse_args()
# yaml のパスと出力先パス
cfg_path    = CONFIG_DIR / f"{args.name}.yaml"
output_path = OUTPUT_DIR / args.name

# UTF-8 で読み込んで Config を生成
with cfg_path.open(encoding="utf-8") as f:
    cfg_dict = yaml.safe_load(f)

config = Config(**cfg_dict,
                output_path=output_path,
                input_path=INPUT_DIR)


# 出力ディレクトリ作成
output_path.mkdir(parents=True, exist_ok=True)

# ログの設定
logger = getLogger(__name__)
logger.setLevel(INFO)
handler = FileHandler(filename=config.output_path / "result.log", encoding="utf-8")
logger.addHandler(handler)
# UTF-8 で読み込んで Config を生成
with cfg_path.open(encoding="utf-8") as f:
    cfg_dict = yaml.safe_load(f)

train_df, test_df = load_data(config=config)
train_df

['fBodyAcc-bandsEnergy()-1,8', 'fBodyAcc-bandsEnergy()-9,16', 'fBodyAcc-bandsEnergy()-17,24', 'fBodyAcc-bandsEnergy()-25,32', 'fBodyAcc-bandsEnergy()-33,40', 'fBodyAcc-bandsEnergy()-41,48', 'fBodyAcc-bandsEnergy()-49,56', 'fBodyAcc-bandsEnergy()-57,64', 'fBodyAcc-bandsEnergy()-1,16', 'fBodyAcc-bandsEnergy()-17,32', 'fBodyAcc-bandsEnergy()-33,48', 'fBodyAcc-bandsEnergy()-49,64', 'fBodyAcc-bandsEnergy()-1,24', 'fBodyAcc-bandsEnergy()-25,48', 'fBodyAccJerk-bandsEnergy()-1,8', 'fBodyAccJerk-bandsEnergy()-9,16', 'fBodyAccJerk-bandsEnergy()-17,24', 'fBodyAccJerk-bandsEnergy()-25,32', 'fBodyAccJerk-bandsEnergy()-33,40', 'fBodyAccJerk-bandsEnergy()-41,48', 'fBodyAccJerk-bandsEnergy()-49,56', 'fBodyAccJerk-bandsEnergy()-57,64', 'fBodyAccJerk-bandsEnergy()-1,16', 'fBodyAccJerk-bandsEnergy()-17,32', 'fBodyAccJerk-bandsEnergy()-33,48', 'fBodyAccJerk-bandsEnergy()-49,64', 'fBodyAccJerk-bandsEnergy()-1,24', 'fBodyAccJerk-bandsEnergy()-25,48', 'fBodyGyro-bandsEnergy()-1,8', 'fBodyGyro-bandsEnergy()-9

C:\Users\sueya\Git-Repositories\takano_labo\dca_yanagi\src\load_data.py:73: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv(root / split / f"X_{split}.txt", delim_whitespace=True, header=None, names=features)
C:\Users\sueya\Git-Repositories\takano_labo\dca_yanagi\src\load_data.py:73: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv(root / split / f"X_{split}.txt", delim_whitespace=True, header=None, names=features)


,subject,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",target
0,5,0.260480,-0.032310,-0.066292,-0.976873,-0.779216,-0.874830,-0.978905,-0.779862,-0.877856,...,0.201887,-0.188315,0.087804,0.020336,0.085813,-0.361440,-0.908870,0.141712,-0.030348,5
1,30,0.275524,-0.014530,-0.094176,-0.997408,-0.969633,-0.988884,-0.998142,-0.970498,-0.989849,...,-0.381182,-0.808719,0.030250,0.238178,0.707377,0.885212,-0.934482,0.117491,-0.026064,5
2,12,0.284224,0.001572,-0.099114,-0.099672,-0.094548,-0.578631,-0.155997,-0.153726,-0.576249,...,-0.083218,-0.480254,0.003401,-0.944625,-0.700109,0.015062,-0.872035,0.185065,0.035282,1
3,1,0.237154,0.007825,-0.122838,-0.979954,-0.866193,-0.968290,-0.980179,-0.882316,-0.966097,...,-0.444482,-0.816502,0.321250,-0.050328,0.584280,-0.426198,-0.654356,0.006141,-0.212753,4
4,1,0.285415,0.016834,-0.106884,-0.324009,0.129619,-0.012624,-0.354572,0.098502,0.073785,...,-0.732537,-0.960982,0.192449,-0.317324,-0.949011,0.560496,-0.609545,0.361467,0.075794,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5053,3,0.262230,-0.016215,-0.139427,-0.416786,-0.106049,-0.312882,-0.449481,-0.124725,-0.277529,...,0.244469,-0.074119,0.065696,0.137927,0.972675,-0.551858,-0.575794,0.351126,0.165120,2
5054,24,0.258587,-0.006958,-0.094029,-0.485825,-0.255627,-0.287639,-0.494420,-0.262824,-0.292375,...,0.395960,0.205011,0.449331,-0.357677,0.973246,-0.461628,-0.808655,0.180537,0.125777,1
5055,3,0.281441,0.022608,-0.005444,-0.342525,-0.080141,-0.419493,-0.372983,-0.097087,-0.438115,...,0.022938,-0.327122,0.140679,-0.830736,-0.749077,0.246029,-0.730683,0.262451,0.107399,1
5056,17,0.269284,-0.019122,-0.119044,-0.993520,-0.971380,-0.981386,-0.993374,-0.969525,-0.980839,...,-0.808637,-0.955617,0.031545,-0.104858,0.336990,0.435159,-0.595726,0.225426,0.274263,5


In [2]:
len(train_df.columns)

563

In [3]:
train_df

,subject,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",target
0,5,0.260480,-0.032310,-0.066292,-0.976873,-0.779216,-0.874830,-0.978905,-0.779862,-0.877856,...,0.201887,-0.188315,0.087804,0.020336,0.085813,-0.361440,-0.908870,0.141712,-0.030348,5
1,30,0.275524,-0.014530,-0.094176,-0.997408,-0.969633,-0.988884,-0.998142,-0.970498,-0.989849,...,-0.381182,-0.808719,0.030250,0.238178,0.707377,0.885212,-0.934482,0.117491,-0.026064,5
2,12,0.284224,0.001572,-0.099114,-0.099672,-0.094548,-0.578631,-0.155997,-0.153726,-0.576249,...,-0.083218,-0.480254,0.003401,-0.944625,-0.700109,0.015062,-0.872035,0.185065,0.035282,1
3,1,0.237154,0.007825,-0.122838,-0.979954,-0.866193,-0.968290,-0.980179,-0.882316,-0.966097,...,-0.444482,-0.816502,0.321250,-0.050328,0.584280,-0.426198,-0.654356,0.006141,-0.212753,4
4,1,0.285415,0.016834,-0.106884,-0.324009,0.129619,-0.012624,-0.354572,0.098502,0.073785,...,-0.732537,-0.960982,0.192449,-0.317324,-0.949011,0.560496,-0.609545,0.361467,0.075794,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5053,3,0.262230,-0.016215,-0.139427,-0.416786,-0.106049,-0.312882,-0.449481,-0.124725,-0.277529,...,0.244469,-0.074119,0.065696,0.137927,0.972675,-0.551858,-0.575794,0.351126,0.165120,2
5054,24,0.258587,-0.006958,-0.094029,-0.485825,-0.255627,-0.287639,-0.494420,-0.262824,-0.292375,...,0.395960,0.205011,0.449331,-0.357677,0.973246,-0.461628,-0.808655,0.180537,0.125777,1
5055,3,0.281441,0.022608,-0.005444,-0.342525,-0.080141,-0.419493,-0.372983,-0.097087,-0.438115,...,0.022938,-0.327122,0.140679,-0.830736,-0.749077,0.246029,-0.730683,0.262451,0.107399,1
5056,17,0.269284,-0.019122,-0.119044,-0.993520,-0.971380,-0.981386,-0.993374,-0.969525,-0.980839,...,-0.808637,-0.955617,0.031545,-0.104858,0.336990,0.435159,-0.595726,0.225426,0.274263,5


In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5058 entries, 0 to 5057
Columns: 563 entries, subject to target
dtypes: float64(561), int64(2)
memory usage: 21.7 MB


In [5]:
#train_df, test_df = load_data_har(config=config)

In [6]:
train_df.columns

Index(['subject', 'tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y',
       'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y',
       'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y',
       'tBodyAcc-mad()-Z',
       ...
       'fBodyBodyGyroJerkMag-skewness()', 'fBodyBodyGyroJerkMag-kurtosis()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'target'],
      dtype='object', length=563)

In [7]:
# インスタンスの生成
data_collaboration = DataCollaborationAnalysis(config=config, logger=logger, train_df=train_df, test_df=test_df)

# データ分割 -> 統合表現の獲得まで一気に実行
data_collaboration.run()

********************データの分割********************


  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 562.86it/s]


num_row 3000 num_col 562
anchor [[0.11505457 0.60906654 0.13339096 ... 0.69726263 0.55644689 0.48987188]
 [0.43338143 0.5737959  0.69868189 ... 0.73024855 0.22954656 0.32172402]
 [0.95244498 0.4640273  0.895524   ... 0.38350442 0.85120652 0.33425618]
 ...
 [0.77341385 0.97239333 0.79854762 ... 0.67170573 0.67559747 0.9441987 ]
 [0.28101504 0.59944565 0.94637381 ... 0.76464201 0.8737928  0.78296077]
 [0.40507762 0.69925217 0.63367146 ... 0.08808387 0.14624683 0.09198194]]
Xs_train[0].shape (562, 562) Xs_test[0].shape (562, 562)
********************中間表現の生成********************
Config(name='exp004', dataset='har', num_institution=9, num_institution_user=562, num_anchor_data=3000, dim_intermediate=562, dim_integrate=562, y_name='target', h_model='random_forest', output_path=WindowsPath('C:/Users/sueya/Git-Repositories/takano_labo/dca_yanagi/output/exp004'), input_path=WindowsPath('C:/Users/sueya/Git-Repositories/takano_labo/dca_yanagi/input'), seed=43)
self.config.dim_intermediate 562



100%|██████████| 9/9 [00:04<00:00,  1.82it/s]


中間表現の次元数:  562
********************統合表現の生成********************


100%|██████████| 9/9 [00:05<00:00,  1.64it/s]

統合表現の次元数:  562


In [8]:
# 集中解析
centralize_analysis(config, logger, y_name=config.y_name)

# 個別解析
individual_analysis(
config=config,
logger=logger,
Xs_train=data_collaboration.Xs_train,
ys_train=data_collaboration.ys_train,
Xs_test=data_collaboration.Xs_test,
ys_test=data_collaboration.ys_test,

)

# 提案手法
dca_analysis(
    X_train_integ=data_collaboration.X_train_integ,
    X_test_integ=data_collaboration.X_test_integ,
    y_train_integ=data_collaboration.y_train_integ,
    y_test_integ=data_collaboration.y_test_integ,
    config=config,
    logger=logger,
)
# train_df, test_df = load_data(config=config)
# y_name = config.y_name
# y_train = train_df.pop(y_name).values
# y_test = test_df.pop(y_name).values

# X_train = train_df.values
# X_test = test_df.values
# dca_analysis(
#     X_train_integ=X_train,
#     y_train_integ=y_train,
#     X_test_integ=X_test,
#     y_test_integ=y_test,
#     config=config,
#     logger=logger,
# )



In [9]:
data_collaboration.X_train_integ

array([[-0.02199168, -0.00193741,  0.03037781, ...,  0.02089628,
        -0.02949354, -0.0365137 ],
       [-0.0215623 ,  0.1480495 ,  0.00143068, ...,  0.09192154,
        -0.17395793, -0.13523286],
       [-0.01261774,  0.06383755, -0.01167625, ...,  0.02984417,
        -0.07177247, -0.08207228],
       ...,
       [-0.01445803,  0.00931933,  0.01107408, ...,  0.02749681,
        -0.00056695, -0.01858404],
       [-0.0222488 ,  0.07548985,  0.01232103, ...,  0.07996689,
        -0.07128218, -0.06238973],
       [-0.00951358,  0.12734982, -0.0256818 , ...,  0.14801013,
        -0.106917  , -0.0707165 ]])

In [10]:
 train_df

,subject,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",target
0,5,0.260480,-0.032310,-0.066292,-0.976873,-0.779216,-0.874830,-0.978905,-0.779862,-0.877856,...,0.201887,-0.188315,0.087804,0.020336,0.085813,-0.361440,-0.908870,0.141712,-0.030348,5
1,30,0.275524,-0.014530,-0.094176,-0.997408,-0.969633,-0.988884,-0.998142,-0.970498,-0.989849,...,-0.381182,-0.808719,0.030250,0.238178,0.707377,0.885212,-0.934482,0.117491,-0.026064,5
2,12,0.284224,0.001572,-0.099114,-0.099672,-0.094548,-0.578631,-0.155997,-0.153726,-0.576249,...,-0.083218,-0.480254,0.003401,-0.944625,-0.700109,0.015062,-0.872035,0.185065,0.035282,1
3,1,0.237154,0.007825,-0.122838,-0.979954,-0.866193,-0.968290,-0.980179,-0.882316,-0.966097,...,-0.444482,-0.816502,0.321250,-0.050328,0.584280,-0.426198,-0.654356,0.006141,-0.212753,4
4,1,0.285415,0.016834,-0.106884,-0.324009,0.129619,-0.012624,-0.354572,0.098502,0.073785,...,-0.732537,-0.960982,0.192449,-0.317324,-0.949011,0.560496,-0.609545,0.361467,0.075794,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5053,3,0.262230,-0.016215,-0.139427,-0.416786,-0.106049,-0.312882,-0.449481,-0.124725,-0.277529,...,0.244469,-0.074119,0.065696,0.137927,0.972675,-0.551858,-0.575794,0.351126,0.165120,2
5054,24,0.258587,-0.006958,-0.094029,-0.485825,-0.255627,-0.287639,-0.494420,-0.262824,-0.292375,...,0.395960,0.205011,0.449331,-0.357677,0.973246,-0.461628,-0.808655,0.180537,0.125777,1
5055,3,0.281441,0.022608,-0.005444,-0.342525,-0.080141,-0.419493,-0.372983,-0.097087,-0.438115,...,0.022938,-0.327122,0.140679,-0.830736,-0.749077,0.246029,-0.730683,0.262451,0.107399,1
5056,17,0.269284,-0.019122,-0.119044,-0.993520,-0.971380,-0.981386,-0.993374,-0.969525,-0.980839,...,-0.808637,-0.955617,0.031545,-0.104858,0.336990,0.435159,-0.595726,0.225426,0.274263,5


In [11]:
train_df

,subject,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",target
0,5,0.260480,-0.032310,-0.066292,-0.976873,-0.779216,-0.874830,-0.978905,-0.779862,-0.877856,...,0.201887,-0.188315,0.087804,0.020336,0.085813,-0.361440,-0.908870,0.141712,-0.030348,5
1,30,0.275524,-0.014530,-0.094176,-0.997408,-0.969633,-0.988884,-0.998142,-0.970498,-0.989849,...,-0.381182,-0.808719,0.030250,0.238178,0.707377,0.885212,-0.934482,0.117491,-0.026064,5
2,12,0.284224,0.001572,-0.099114,-0.099672,-0.094548,-0.578631,-0.155997,-0.153726,-0.576249,...,-0.083218,-0.480254,0.003401,-0.944625,-0.700109,0.015062,-0.872035,0.185065,0.035282,1
3,1,0.237154,0.007825,-0.122838,-0.979954,-0.866193,-0.968290,-0.980179,-0.882316,-0.966097,...,-0.444482,-0.816502,0.321250,-0.050328,0.584280,-0.426198,-0.654356,0.006141,-0.212753,4
4,1,0.285415,0.016834,-0.106884,-0.324009,0.129619,-0.012624,-0.354572,0.098502,0.073785,...,-0.732537,-0.960982,0.192449,-0.317324,-0.949011,0.560496,-0.609545,0.361467,0.075794,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5053,3,0.262230,-0.016215,-0.139427,-0.416786,-0.106049,-0.312882,-0.449481,-0.124725,-0.277529,...,0.244469,-0.074119,0.065696,0.137927,0.972675,-0.551858,-0.575794,0.351126,0.165120,2
5054,24,0.258587,-0.006958,-0.094029,-0.485825,-0.255627,-0.287639,-0.494420,-0.262824,-0.292375,...,0.395960,0.205011,0.449331,-0.357677,0.973246,-0.461628,-0.808655,0.180537,0.125777,1
5055,3,0.281441,0.022608,-0.005444,-0.342525,-0.080141,-0.419493,-0.372983,-0.097087,-0.438115,...,0.022938,-0.327122,0.140679,-0.830736,-0.749077,0.246029,-0.730683,0.262451,0.107399,1
5056,17,0.269284,-0.019122,-0.119044,-0.993520,-0.971380,-0.981386,-0.993374,-0.969525,-0.980839,...,-0.808637,-0.955617,0.031545,-0.104858,0.336990,0.435159,-0.595726,0.225426,0.274263,5
